In [1]:
import pandas as pd
import numpy as np 

In [166]:
#Loading pickle of solutions
Final_Diet_It = pd.read_pickle("data/processed/Italy_opt_diet.pkl")
Final_Diet_Ge = pd.read_pickle("data/processed/Germany_opt_diet.pkl")
Final_Diet_Fr = pd.read_pickle("data/processed/France_opt_diet.pkl")
Final_Diet_Sp = pd.read_pickle("data/processed/Spain_opt_diet.pkl")
Final_Diet_UK = pd.read_pickle("data/processed/United Kingdom_opt_diet.pkl")
Final_Diet_EU = Final_Diet_It + Final_Diet_Ge + Final_Diet_Fr + Final_Diet_Sp + Final_Diet_UK

In [267]:
import plotly.graph_objects as go
import plotly.offline as py
def build_hierarchical_dataframe(df, levels, value_column, color_columns=None, prod=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy, 
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'label', 'parent', 'value', 'log_val','color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value','log_val', 'color'])
        dfg = df.groupby(levels[i:]).sum(numerical_only=True)
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if(i==0):
            df_tree['label'] = dfg['Product'].copy()
        else:
            df_tree['label'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'Total'
        df_tree['log_val'] = np.log(dfg[value_column]+2)
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='Total', parent='', 
                              value=df_tree['value'].sum(),
                              color=df_tree['color'].sum(), 
                              label="Total",
#                               log_val=np.log1p(df[value_column].sum())))
                              log_val=sum(df_all_trees[df_all_trees.id.isin(prod)]['log_val'])))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    
    return df_all_trees

In [312]:
def make_sunburst(dataset, path, EU=False):
    df_val = dataset.copy()
    prod = dataset.index.values
    df_val = df_val.apply(pd.to_numeric)
    df_val = df_val.stack().to_frame().reset_index()
    df_val = df_val.rename(columns = {"level_1": "af_countries", 0:"val"})
    levels = ['id', 'af_countries'] # levels used for the hierarchical chart
    color_columns = ['val']
    value_column = 'val'
    id_n = df_val.index.values
    df_val['id'] = df_val.Product + id_n.astype(str)
    df_val = df_val[['Product', 'id', 'af_countries', 'val']]
    df_all_trees = build_hierarchical_dataframe(df_val, levels, value_column, color_columns, prod)
    df_all_trees = df_all_trees.replace({'United Republic of Tanzania':'Tanzania', 'Central African Republic':'Centr. Afr. Rep.'})
    
    if (EU):
        color_scale = "YlGnBu"
    else:
        color_scale = "RdBu"
    fig =go.Figure(go.Sunburst(
        ids=df_all_trees.id,
        labels=df_all_trees.label,
        parents=df_all_trees.parent,
        maxdepth=2,
        values=df_all_trees.log_val,
        customdata = df_all_trees.value,
        hovertemplate='<b>%{label}</b><br><br>Need: %{customdata:.3f} tons',
        name='',
        marker = dict(colorscale=color_scale, line=dict(color="#777", width=0.5)),
#         branchvalues='total'
    ))
    fig.update_layout(margin = dict(t=0, l=0, r=0, b=20), paper_bgcolor = "rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
    py.plot(fig, filename=path)

In [313]:
make_sunburst(Final_Diet_EU, "docs/_includes/sunburst_EU.html", EU=True)

In [314]:
make_sunburst(Final_Diet_It, "docs/_includes/sunburst_it.html")
make_sunburst(Final_Diet_Fr, "docs/_includes/sunburst_fr.html")
make_sunburst(Final_Diet_Ge, "docs/_includes/sunburst_ge.html")
make_sunburst(Final_Diet_Sp, "docs/_includes/sunburst_sp.html")
make_sunburst(Final_Diet_UK, "docs/_includes/sunburst_uk.html")